# Electricity Consumption Time Series Forecasting - Complete Pipeline

## Project Overview

This notebook implements a comprehensive time series forecasting pipeline for predicting household electricity consumption. The project follows industry best practices and includes:

1. **Data Loading and Preprocessing**: Handling missing values, resampling to hourly frequency
2. **Exploratory Data Analysis**: Time series decomposition, stationarity tests, autocorrelation analysis
3. **Feature Engineering**: Temporal features, lag features, rolling statistics
4. **Model Development**: Classical (ARIMA, SARIMA, ETS, STL), ML (XGBoost, Random Forest, Linear Regression), and Advanced (LSTM, GRU, Prophet) models
5. **Model Evaluation**: Comprehensive metrics and visualizations
6. **Model Comparison**: Statistical comparison and best model selection

## Objective

Forecast the next hour's average Global_active_power consumption using minute-level electricity consumption data.



## 1. Setup and Imports

Import all necessary libraries and set up the environment. We'll use our modular code structure for clean, maintainable code.


In [ ]:
# Standard library imports
import sys
import os
import warnings
warnings.filterwarnings('ignore')

# Add src to path for imports
sys.path.append('../')

# Data processing
import pandas as pd
import numpy as np

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

# Time series analysis
from statsmodels.tsa.stattools import adfuller, kpss
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from scipy import stats

# Project modules
from src.data.data_loader import load_data
from src.data.data_preprocessor import preprocess_data, resample_to_hourly, handle_missing_values, combine_datetime
from src.data.data_validator import validate_data
from src.features.feature_engineer import create_all_features
from src.features.feature_selection import select_features
from src.validation.time_series_split import time_series_split
from src.models.classical_models import ARIMAModel, SARIMAModel, ETSModel, STLModel
from src.models.ml_models import XGBoostModel, RandomForestModel, LinearRegressionModel
from src.models.dl_models import LSTMModel, GRUModel, ProphetModel
from src.evaluation.metrics import calculate_metrics, print_metrics
from src.evaluation.visualizations import plot_forecast, plot_residuals, plot_model_comparison
from src.evaluation.model_comparison import compare_models, select_best_model, generate_evaluation_report
from src.utils.config import Config
from src.utils.logger import setup_logger

# Set random seeds for reproducibility
np.random.seed(42)

# Configuration
config = Config()
logger = setup_logger(log_file=config.LOG_FILE, log_level=config.LOG_LEVEL)

print("✓ All imports successful!")
print(f"✓ Configuration loaded: Train split = {config.TRAIN_SPLIT*100:.0f}%, Forecast horizon = {config.FORECAST_HORIZON} hours")


## 2. Data Loading

Load the household power consumption dataset. The data is in semicolon-separated format with "?" representing missing values. We'll parse it properly and handle the date/time columns.


In [ ]:
# Load the data
# Note: For faster execution during development, you can use nrows parameter
# For full analysis, set nrows=None
df_raw = load_data(config.DATA_PATH, nrows=None)

print(f"Data shape: {df_raw.shape}")
print(f"\nColumn names: {list(df_raw.columns)}")
print(f"\nData types:\n{df_raw.dtypes}")
print(f"\nFirst few rows:")
df_raw.head()


In [ ]:
# Check for missing values and data quality
print("Missing values per column:")
print(df_raw.isnull().sum())
print(f"\nTotal missing values: {df_raw.isnull().sum().sum()}")
print(f"\nDate range in raw data:")
print(f"  Date column range: {df_raw['Date'].min()} to {df_raw['Date'].max()}")
print(f"  Time column range: {df_raw['Time'].min()} to {df_raw['Time'].max()}")

# Basic statistics
print("\nBasic statistics:")
df_raw.describe()


## 3. Data Preprocessing

This step is critical for time series analysis:
1. **Combine Date and Time**: Create a single datetime index
2. **Handle Missing Values**: Use interpolation (appropriate for time series)
3. **Resample to Hourly**: Aggregate minute-level data to hourly using mean
4. **Outlier Treatment**: Detect and cap extreme outliers
5. **Data Validation**: Ensure chronological order and data quality

**Justification**: Time series models require continuous, evenly-spaced data. Missing values break continuity, and outliers can distort model training. Hourly aggregation reduces noise while preserving important patterns.


In [ ]:
# Complete preprocessing pipeline
df_processed = preprocess_data(
    df_raw,
    date_col='Date',
    time_col='Time',
    missing_method='interpolate',  # Interpolation preserves temporal patterns
    outlier_method='cap',  # Cap outliers rather than remove to preserve data continuity
    resample_method='mean',  # Mean aggregation for hourly data
    target_col='Global_active_power'
)

print(f"Processed data shape: {df_processed.shape}")
print(f"Date range: {df_processed.index.min()} to {df_processed.index.max()}")
print(f"Frequency: {pd.infer_freq(df_processed.index)}")
print(f"\nMissing values after preprocessing: {df_processed.isnull().sum().sum()}")

# Validate the processed data
is_valid, issues = validate_data(df_processed, target_col='Global_active_power')
if is_valid:
    print("\n✓ Data validation passed!")
else:
    print(f"\n⚠ Data validation issues: {issues}")

df_processed.head()


## 4. Exploratory Data Analysis (EDA)

Comprehensive EDA helps us understand:
- **Trends**: Long-term patterns in consumption
- **Seasonality**: Daily, weekly, monthly patterns
- **Stationarity**: Whether the series is stationary (required for some models)
- **Autocorrelation**: Relationships between past and current values

**Justification**: Understanding these characteristics guides model selection. For example, strong seasonality suggests SARIMA or Prophet, while non-stationarity requires differencing.


In [ ]:
# Plot the time series
target_col = 'Global_active_power'
y = df_processed[target_col]

plt.figure(figsize=(16, 6))
plt.plot(y.index, y.values, linewidth=0.5, alpha=0.7)
plt.title('Global Active Power Consumption Over Time', fontsize=14, fontweight='bold')
plt.xlabel('Date', fontsize=12)
plt.ylabel('Global Active Power (kW)', fontsize=12)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# Summary statistics
print("Time Series Summary Statistics:")
print(f"  Mean: {y.mean():.4f} kW")
print(f"  Std:  {y.std():.4f} kW")
print(f"  Min:  {y.min():.4f} kW")
print(f"  Max:  {y.max():.4f} kW")
print(f"  Total observations: {len(y)}")


In [ ]:
# Time series decomposition
# Multiplicative or additive - we'll try additive first
decomposition = seasonal_decompose(y, model='additive', period=24)  # 24-hour period

fig, axes = plt.subplots(4, 1, figsize=(16, 12))
decomposition.observed.plot(ax=axes[0], title='Original', color='blue')
decomposition.trend.plot(ax=axes[1], title='Trend', color='green')
decomposition.seasonal.plot(ax=axes[2], title='Seasonal (24-hour)', color='orange')
decomposition.resid.plot(ax=axes[3], title='Residual', color='red')

for ax in axes:
    ax.set_ylabel('Power (kW)', fontsize=10)
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\nDecomposition Summary:")
print(f"  Trend range: {decomposition.trend.min():.2f} to {decomposition.trend.max():.2f}")
print(f"  Seasonal range: {decomposition.seasonal.min():.2f} to {decomposition.seasonal.max():.2f}")
print(f"  Residual std: {decomposition.resid.std():.4f}")


In [ ]:
# Stationarity tests
print("STATIONARITY TESTS")
print("=" * 60)

# Augmented Dickey-Fuller test
adf_result = adfuller(y.dropna())
print("\n1. Augmented Dickey-Fuller (ADF) Test:")
print(f"   ADF Statistic: {adf_result[0]:.4f}")
print(f"   p-value: {adf_result[1]:.4f}")
print(f"   Critical Values:")
for key, value in adf_result[4].items():
    print(f"      {key}: {value:.4f}")
if adf_result[1] <= 0.05:
    print("   ✓ Series is STATIONARY (p < 0.05)")
else:
    print("   ✗ Series is NON-STATIONARY (p >= 0.05)")
    print("   → Differencing may be required for ARIMA models")

# KPSS test
kpss_result = kpss(y.dropna(), regression='c')
print("\n2. KPSS Test:")
print(f"   KPSS Statistic: {kpss_result[0]:.4f}")
print(f"   p-value: {kpss_result[1]:.4f}")
print(f"   Critical Values:")
for key, value in kpss_result[3].items():
    print(f"      {key}: {value:.4f}")
if kpss_result[1] >= 0.05:
    print("   ✓ Series is STATIONARY (p >= 0.05)")
else:
    print("   ✗ Series is NON-STATIONARY (p < 0.05)")


In [ ]:
# Autocorrelation analysis
fig, axes = plt.subplots(2, 1, figsize=(14, 10))

# ACF plot
plot_acf(y.dropna(), lags=48, ax=axes[0], title='Autocorrelation Function (ACF)', alpha=0.05)
axes[0].set_xlabel('Lag (hours)', fontsize=11)
axes[0].set_ylabel('ACF', fontsize=11)
axes[0].grid(True, alpha=0.3)

# PACF plot
plot_pacf(y.dropna(), lags=48, ax=axes[1], title='Partial Autocorrelation Function (PACF)', alpha=0.05, method='ywm')
axes[1].set_xlabel('Lag (hours)', fontsize=11)
axes[1].set_ylabel('PACF', fontsize=11)
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\nACF/PACF Analysis:")
print("  - Significant lags in ACF indicate moving average (MA) components")
print("  - Significant lags in PACF indicate autoregressive (AR) components")
print("  - Strong seasonal patterns (24h, 168h) suggest SARIMA models")


In [ ]:
# Seasonal patterns analysis
df_processed['hour'] = df_processed.index.hour
df_processed['day_of_week'] = df_processed.index.dayofweek
df_processed['month'] = df_processed.index.month

# Hourly pattern
hourly_avg = df_processed.groupby('hour')[target_col].mean()
# Day of week pattern
dow_avg = df_processed.groupby('day_of_week')[target_col].mean()
# Monthly pattern
monthly_avg = df_processed.groupby('month')[target_col].mean()

fig, axes = plt.subplots(1, 3, figsize=(18, 5))

hourly_avg.plot(kind='bar', ax=axes[0], color='steelblue', edgecolor='black')
axes[0].set_title('Average Consumption by Hour of Day', fontweight='bold')
axes[0].set_xlabel('Hour', fontsize=11)
axes[0].set_ylabel('Average Power (kW)', fontsize=11)
axes[0].grid(True, alpha=0.3, axis='y')

dow_avg.plot(kind='bar', ax=axes[1], color='coral', edgecolor='black')
axes[1].set_title('Average Consumption by Day of Week', fontweight='bold')
axes[1].set_xlabel('Day of Week (0=Mon, 6=Sun)', fontsize=11)
axes[1].set_ylabel('Average Power (kW)', fontsize=11)
axes[1].grid(True, alpha=0.3, axis='y')

monthly_avg.plot(kind='bar', ax=axes[2], color='green', edgecolor='black')
axes[2].set_title('Average Consumption by Month', fontweight='bold')
axes[2].set_xlabel('Month', fontsize=11)
axes[2].set_ylabel('Average Power (kW)', fontsize=11)
axes[2].grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

print("\nSeasonal Patterns:")
print(f"  Hourly range: {hourly_avg.min():.2f} to {hourly_avg.max():.2f} kW")
print(f"  Day of week range: {dow_avg.min():.2f} to {dow_avg.max():.2f} kW")
print(f"  Monthly range: {monthly_avg.min():.2f} to {monthly_avg.max():.2f} kW")


In [ ]:
# Create all features
df_features = create_all_features(
    df_processed,
    target_col=target_col,
    lags=config.LAG_FEATURES,
    rolling_windows=config.ROLLING_WINDOWS
)

print(f"Original features: {len(df_processed.columns)}")
print(f"Total features after engineering: {len(df_features.columns)}")
print(f"\nNew feature categories:")
print(f"  Temporal features: {len([c for c in df_features.columns if any(x in c for x in ['hour', 'day', 'month', 'year', 'week', 'sin', 'cos'])])}")
print(f"  Lag features: {len([c for c in df_features.columns if 'lag' in c])}")
print(f"  Rolling features: {len([c for c in df_features.columns if 'rolling' in c or 'ewm' in c])}")
print(f"  Derived features: {len([c for c in df_features.columns if any(x in c for x in ['diff', 'change', 'power_factor', 'apparent'])])}")

# Remove temporary columns added during EDA
if 'hour' in df_features.columns:
    df_features = df_features.drop(columns=['hour', 'day_of_week', 'month'])

df_features.head()


In [ ]:
# Select features for ML models
# Remove rows with NaN from lag/rolling features (first few rows)
df_features = df_features.dropna()

# Select features (exclude target)
feature_cols = select_features(
    df_features,
    target_col=target_col,
    method='correlation',
    threshold=0.05  # Include features with at least 5% correlation
)

print(f"Selected {len(feature_cols)} features for ML models")
print(f"\nTop 10 features by correlation:")
feature_corr = df_features[feature_cols + [target_col]].corr()[target_col].abs().sort_values(ascending=False)
print(feature_corr.head(10))


## 6. Train-Test Split

**Critical**: Time series data must be split chronologically, not randomly. Future data should never be used to predict the past.

**Justification**: Random splits cause data leakage and unrealistic performance estimates. Chronological splits simulate real-world forecasting where we predict future using only past data.


In [ ]:
# Split data chronologically
train_df, test_df = time_series_split(
    df_features,
    train_size=config.TRAIN_SPLIT
)

# Separate target and features
y_train = train_df[target_col]
y_test = test_df[target_col]

# For classical models (work with series only)
y_train_series = y_train.copy()
y_test_series = y_test.copy()

# For ML models (need features)
X_train = train_df[feature_cols]
X_test = test_df[feature_cols]

print(f"Training set: {len(train_df)} samples ({train_df.index.min()} to {train_df.index.max()})")
print(f"Test set: {len(test_df)} samples ({test_df.index.min()} to {test_df.index.max()})")
print(f"\nTraining target statistics:")
print(f"  Mean: {y_train.mean():.4f} kW")
print(f"  Std:  {y_train.std():.4f} kW")
print(f"\nTest target statistics:")
print(f"  Mean: {y_test.mean():.4f} kW")
print(f"  Std:  {y_test.std():.4f} kW")


## 7. Model Training and Evaluation

We'll train multiple model types and compare their performance:

1. **Classical Models**: ARIMA, SARIMA, ETS, STL
2. **Machine Learning Models**: XGBoost, Random Forest, Linear Regression
3. **Advanced Models**: LSTM, GRU, Prophet

**Justification**: Different models capture different patterns. Classical models are interpretable and work well with strong seasonality. ML models can capture complex non-linear patterns. Deep learning models can learn long-term dependencies.


In [ ]:
# Store all model results
all_metrics = {}
all_predictions = {}

# Forecast horizon
forecast_horizon = config.FORECAST_HORIZON


### 7.1 Classical Time Series Models


In [ ]:
# ARIMA Model
print("Training ARIMA model...")
try:
    arima_model = ARIMAModel(use_auto=True)
    arima_model.fit(y_train_series)
    arima_pred = arima_model.predict(n_periods=len(y_test))
    arima_metrics = calculate_metrics(y_test.values, arima_pred)
    all_metrics['ARIMA'] = arima_metrics
    all_predictions['ARIMA'] = arima_pred
    print_metrics(arima_metrics, "ARIMA")
except Exception as e:
    print(f"ARIMA model failed: {e}")
    all_metrics['ARIMA'] = {}


In [ ]:
# SARIMA Model (with 24-hour seasonality)
print("Training SARIMA model...")
try:
    sarima_model = SARIMAModel(
        order=(1, 1, 1),
        seasonal_order=(1, 1, 1, 24),  # 24-hour seasonality
        use_auto=True
    )
    sarima_model.fit(y_train_series)
    sarima_pred = sarima_model.predict(n_periods=len(y_test))
    sarima_metrics = calculate_metrics(y_test.values, sarima_pred)
    all_metrics['SARIMA'] = sarima_metrics
    all_predictions['SARIMA'] = sarima_pred
    print_metrics(sarima_metrics, "SARIMA")
except Exception as e:
    print(f"SARIMA model failed: {e}")
    all_metrics['SARIMA'] = {}


In [ ]:
# ETS Model (Exponential Smoothing)
print("Training ETS model...")
try:
    ets_model = ETSModel(trend='add', seasonal='add', seasonal_periods=24)
    ets_model.fit(y_train_series)
    ets_pred = ets_model.predict(n_periods=len(y_test))
    ets_metrics = calculate_metrics(y_test.values, ets_pred)
    all_metrics['ETS'] = ets_metrics
    all_predictions['ETS'] = ets_pred
    print_metrics(ets_metrics, "ETS")
except Exception as e:
    print(f"ETS model failed: {e}")
    all_metrics['ETS'] = {}


In [ ]:
# STL Decomposition Model
print("Training STL model...")
try:
    stl_model = STLModel(period=24, robust=True)
    stl_model.fit(y_train_series)
    stl_pred = stl_model.predict(n_periods=len(y_test))
    stl_metrics = calculate_metrics(y_test.values, stl_pred)
    all_metrics['STL'] = stl_metrics
    all_predictions['STL'] = stl_pred
    print_metrics(stl_metrics, "STL")
except Exception as e:
    print(f"STL model failed: {e}")
    all_metrics['STL'] = {}


### 7.2 Machine Learning Models


In [ ]:
# XGBoost Model
print("Training XGBoost model...")
try:
    xgb_model = XGBoostModel(n_estimators=100, max_depth=6, learning_rate=0.1)
    xgb_model.fit(X_train, y_train, tune_hyperparameters=False)
    xgb_pred = xgb_model.predict(X_test)
    xgb_metrics = calculate_metrics(y_test.values, xgb_pred)
    all_metrics['XGBoost'] = xgb_metrics
    all_predictions['XGBoost'] = xgb_pred
    print_metrics(xgb_metrics, "XGBoost")
    
    # Feature importance
    print("\nTop 10 Most Important Features:")
    importance = xgb_model.get_feature_importance().head(10)
    print(importance)
except Exception as e:
    print(f"XGBoost model failed: {e}")
    all_metrics['XGBoost'] = {}


In [ ]:
# Random Forest Model
print("Training Random Forest model...")
try:
    rf_model = RandomForestModel(n_estimators=100, max_depth=10)
    rf_model.fit(X_train, y_train, tune_hyperparameters=False)
    rf_pred = rf_model.predict(X_test)
    rf_metrics = calculate_metrics(y_test.values, rf_pred)
    all_metrics['Random Forest'] = rf_metrics
    all_predictions['Random Forest'] = rf_pred
    print_metrics(rf_metrics, "Random Forest")
except Exception as e:
    print(f"Random Forest model failed: {e}")
    all_metrics['Random Forest'] = {}


In [ ]:
# Linear Regression (Ridge)
print("Training Linear Regression (Ridge) model...")
try:
    lr_model = LinearRegressionModel(model_type='ridge', alpha=1.0)
    lr_model.fit(X_train, y_train, tune_hyperparameters=False)
    lr_pred = lr_model.predict(X_test)
    lr_metrics = calculate_metrics(y_test.values, lr_pred)
    all_metrics['Linear Regression'] = lr_metrics
    all_predictions['Linear Regression'] = lr_pred
    print_metrics(lr_metrics, "Linear Regression")
except Exception as e:
    print(f"Linear Regression model failed: {e}")
    all_metrics['Linear Regression'] = {}


### 7.3 Advanced Models

**Note**: Deep learning models (LSTM/GRU) and Prophet may take longer to train. They are optional but provide state-of-the-art performance.


In [ ]:
# Prophet Model (Facebook's time series model)
print("Training Prophet model...")
try:
    prophet_model = ProphetModel(daily_seasonality=True, weekly_seasonality=True, yearly_seasonality=True)
    prophet_model.fit(y_train_series)
    prophet_forecast = prophet_model.predict(n_periods=len(y_test), freq='H')
    prophet_pred = prophet_model.get_forecast_values(prophet_forecast)
    prophet_metrics = calculate_metrics(y_test.values, prophet_pred)
    all_metrics['Prophet'] = prophet_metrics
    all_predictions['Prophet'] = prophet_pred
    print_metrics(prophet_metrics, "Prophet")
except Exception as e:
    print(f"Prophet model failed: {e}")
    all_metrics['Prophet'] = {}


## 8. Model Comparison and Visualization

Compare all models and visualize their performance.


In [ ]:
# Create comparison DataFrame
comparison_df = compare_models(all_metrics, primary_metric='RMSE')
print("\nModel Comparison (sorted by RMSE):")
print(comparison_df)


In [ ]:
# Visualize model comparison
plot_model_comparison(all_metrics, metric_name='RMSE', save_path='outputs/model_comparison_rmse.png')
plot_model_comparison(all_metrics, metric_name='MAE', save_path='outputs/model_comparison_mae.png')
plot_model_comparison(all_metrics, metric_name='MAPE', save_path='outputs/model_comparison_mape.png')


In [ ]:
# Select best model
best_model_name, best_model_metrics = select_best_model(all_metrics, primary_metric='RMSE')
print(f"\n🏆 Best Model: {best_model_name}")
print("\nBest Model Metrics:")
print_metrics(best_model_metrics, best_model_name)

# Plot forecasts for best model
best_pred = all_predictions[best_model_name]
plot_forecast(
    y_test,
    best_pred,
    title=f'Best Model Forecast: {best_model_name}',
    save_path=f'outputs/best_model_forecast_{best_model_name.replace(" ", "_")}.png'
)


In [ ]:
# Plot residuals for best model
plot_residuals(
    y_test.values,
    best_pred,
    title=f'Residual Analysis: {best_model_name}',
    save_path=f'outputs/residuals_{best_model_name.replace(" ", "_")}.png'
)


## 9. Final Forecast and Summary

Generate final forecast for the next 24-48 hours and create evaluation report.


In [ ]:
# Generate final forecast for next 24-48 hours using best model
# Use full dataset for training
y_full = df_features[target_col]

print(f"Generating {config.FORECAST_HORIZON} hour ahead forecast using {best_model_name}...")

# Retrain best model on full data
if best_model_name in ['ARIMA', 'SARIMA', 'ETS', 'STL', 'Prophet']:
    # Classical models
    if best_model_name == 'ARIMA':
        final_model = ARIMAModel(use_auto=True)
        final_model.fit(y_full)
        final_forecast = final_model.predict(n_periods=config.FORECAST_HORIZON)
    elif best_model_name == 'SARIMA':
        final_model = SARIMAModel(use_auto=True)
        final_model.fit(y_full)
        final_forecast = final_model.predict(n_periods=config.FORECAST_HORIZON)
    elif best_model_name == 'ETS':
        final_model = ETSModel(trend='add', seasonal='add', seasonal_periods=24)
        final_model.fit(y_full)
        final_forecast = final_model.predict(n_periods=config.FORECAST_HORIZON)
    elif best_model_name == 'STL':
        final_model = STLModel(period=24, robust=True)
        final_model.fit(y_full)
        final_forecast = final_model.predict(n_periods=config.FORECAST_HORIZON)
    elif best_model_name == 'Prophet':
        final_model = ProphetModel()
        final_model.fit(y_full)
        forecast_df = final_model.predict(n_periods=config.FORECAST_HORIZON, freq='H')
        final_forecast = final_model.get_forecast_values(forecast_df)
else:
    # ML models
    X_full = df_features[feature_cols]
    if best_model_name == 'XGBoost':
        final_model = XGBoostModel()
        final_model.fit(X_full, y_full)
        # For future predictions, we need to create features
        # This is simplified - in production, you'd need to generate future features
        final_forecast = final_model.predict(X_full.tail(config.FORECAST_HORIZON))
    elif best_model_name == 'Random Forest':
        final_model = RandomForestModel()
        final_model.fit(X_full, y_full)
        final_forecast = final_model.predict(X_full.tail(config.FORECAST_HORIZON))
    else:
        final_model = LinearRegressionModel()
        final_model.fit(X_full, y_full)
        final_forecast = final_model.predict(X_full.tail(config.FORECAST_HORIZON))

# Create forecast index
forecast_index = pd.date_range(
    start=y_full.index[-1] + pd.Timedelta(hours=1),
    periods=config.FORECAST_HORIZON,
    freq='H'
)

# Plot final forecast
plt.figure(figsize=(16, 6))
plt.plot(y_full.index[-168:], y_full.values[-168:], label='Historical (last 7 days)', linewidth=2, alpha=0.7)
plt.plot(forecast_index, final_forecast, label=f'Forecast ({best_model_name})', linewidth=2, linestyle='--', color='red')
plt.axvline(x=y_full.index[-1], color='gray', linestyle=':', linewidth=2, label='Forecast Start')
plt.xlabel('Date', fontsize=12)
plt.ylabel('Global Active Power (kW)', fontsize=12)
plt.title(f'{config.FORECAST_HORIZON}-Hour Ahead Forecast using {best_model_name}', fontsize=14, fontweight='bold')
plt.legend(fontsize=11)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig('outputs/final_forecast.png', dpi=300, bbox_inches='tight')
plt.show()

print(f"\n✓ Forecast generated for {config.FORECAST_HORIZON} hours ahead")
print(f"  Forecast range: {forecast_index[0]} to {forecast_index[-1]}")
print(f"  Forecast values range: {final_forecast.min():.2f} to {final_forecast.max():.2f} kW")


In [ ]:
# Generate comprehensive evaluation report
report = generate_evaluation_report(
    all_metrics,
    primary_metric='RMSE',
    save_path='outputs/evaluation_report.txt'
)
print(report)


## 10. Key Findings and Conclusions

### Summary

1. **Data Quality**: Successfully handled 25,979 missing values and resampled to hourly frequency
2. **Patterns Identified**: Strong daily (24h) and weekly (168h) seasonality patterns
3. **Stationarity**: Series shows non-stationary behavior, requiring differencing for ARIMA models
4. **Best Model**: [Will be determined after execution]

### Recommendations

1. **Model Selection**: Use the best performing model based on RMSE/MAPE on test set
2. **Feature Engineering**: Lag features and rolling statistics significantly improve ML model performance
3. **Monitoring**: Regularly retrain models as new data arrives
4. **Ensemble**: Consider ensemble methods combining top-performing models

### Next Steps

1. Deploy best model for production forecasting
2. Set up monitoring and retraining pipeline
3. Implement confidence intervals for uncertainty quantification
4. Explore external features (weather, holidays) if available
